In [12]:
%run Imports.ipynb

In [1]:
def FormatAndPrint(sentence):
    print(str(sentence)[0:62])
    
def FormatDay(i):
    day = str(i)
    if len(day) == 1:
        day = '0'+day
    return day
        
def CountTrade(percentage, predictions, probas = [], portfolio = 10000.00, cash = 5000.00, trade_amount = 500, steps = True):
    total_amount_start = portfolio + cash
    
    for i in tqdm(range(0, len(predictions))):

        if steps:   
            FormatAndPrint("\n")
            FormatAndPrint(f'============================ DAY {FormatDay(i)} ==========================')
            FormatAndPrint(f'==============================================================')
        old_daily_status = cash + portfolio
        old_cash = cash
        old_portfolio = portfolio
        
        if steps:
            FormatAndPrint(f'============================ ACTION ==========================')
            
        if predictions[i] == 'Rise':
            
            # I have enough money for buy
            if cash >= 500:
                cash -=500
                portfolio+=500
                if steps:
                    FormatAndPrint('==============       >I BOUGHT BTC for 500<  =================')
            
            # I have not enough money for buy
            elif cash <500:
                if steps:
                    FormatAndPrint('I havent got enough money ......... BUY action FAILED')
                

        elif predictions[i] == 'Fall':
            
            #My portfolio is enough for sell some BTC
            if portfolio >=500:
                portfolio-=500
                cash += 500
                if steps:
                    FormatAndPrint('==============      >I SELL BTC for 500<     =================')
                
            #My portfolio is too small for sell BTC for 500
            elif portfolio<500:
                if steps:
                    FormatAndPrint('I havent got enough porfolio....... SELL action FAILED')
        
        market_coeficient = 1+percentage[i]
        
        #count new portfolio  
        portfolio = round((portfolio*market_coeficient),2)
        
        if steps:
            FormatAndPrint(f'==============================================================')
            FormatAndPrint(f'===========================  STATUS  =========================')
            FormatAndPrint(f'========== CASH:    {old_cash},-  >>   {cash},-     =======================')
            FormatAndPrint(f'===== PORTFOLIO:   {old_portfolio},- >>   {portfolio},-     ====================')
        
        
        if old_daily_status > cash+portfolio:
            if steps:
                FormatAndPrint(f'======= SUMMARY:   {old_daily_status},- >> {round((cash+portfolio),2)},-  ========================')
        else:
            if steps:
                FormatAndPrint(f'======= SUMMARY:   {old_daily_status},- >> {round((cash+portfolio),2)},-  ========================')
        if steps:
            FormatAndPrint(f'==============================================================')
            FormatAndPrint(f'========================== BALANCE ===========================')
            FormatAndPrint(f'==========================  {round(((cash+portfolio)/old_daily_status)-1,4)*100}%  =============================')
            FormatAndPrint(f'==============================================================')

    FormatAndPrint("\n")
    total_amount_end = round((portfolio + cash),2)
    global_balance = round((total_amount_end/total_amount_start),4)*100
    global_balance = round((global_balance-100),2)
    FormatAndPrint(f'======================= GLOBAL BALANCE ========================')
    FormatAndPrint(f'=========================== {global_balance}% =============================')
    return global_balance

In [187]:
#CountTrade(percentage, predictions)

In [1]:
def SimulateTradeRF(df, no_runs = 100, steps = True, max_depth=7, min_samples_split=5, min_samples_leaf = 1):
    bilance = []
    for i in tqdm(range(0, no_runs)):
        y_test , y_pred, values = Predict(df, target_value=True, scaler = StandardScaler(), classifier = RandomForestClassifier(n_estimators=101, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf = min_samples_leaf ), test_size=0.2)
        values = values/100
        if steps:
            GetSimpleResultMetrics(y_test, y_pred)
        bilance.append(CountTrade(values.values, y_pred, steps=steps ))
    return bilance, len(y_pred)
    

In [3]:
def SimulateTradeOwnModel(df, no_runs = 100, steps = True, max_depth=7, min_samples_split=5, min_samples_leaf = 1):
    df.rename(columns={'target': 'target_value'}, inplace=True)
    df['target'] = np.where(df['target_value']>0, 'Rise', 'Fall')
    bilance = []
    
    for i in tqdm(range(0, no_runs)):
        y_test , y_pred, values = Predict(df, target_value=True, scaler = StandardScaler(), classifier = RandomForestClassifier(n_estimators=101, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf = min_samples_leaf ), test_size=0.2)
        values = values/100
        if steps:
            GetSimpleResultMetrics(y_test, y_pred)
        bilance.append(CountTrade(values.values, y_pred, steps=steps ))
    return bilance, len(y_pred)